# Test notebook for Filtering and PDB manipulation

In [4]:
import sys
import os
import biotite
import biotite.structure as structure
import biotite.structure.io.pdb as pdb
import numpy as np


# Test notebook for Filtering and PDB manipulation

pdb_file = "boltz_results_391/predictions/391/391_model_0.pdb"

# Load the PDB file and parse it
pdb_file_obj = pdb.PDBFile.read(pdb_file)
pdb_data = pdb_file_obj.get_structure()

# Print the structure information
print("PDB Structure Information:")
print(f"Type of structure: {type(pdb_data)}")
print(f"Number of atoms: {len(pdb_data)}")

# Check if it's an AtomArrayStack (multiple models) or AtomArray (single model)
if isinstance(pdb_data, structure.AtomArrayStack):
    print(f"Number of models: {pdb_data.stack_depth()}")
    print(f"Atom array shape: {pdb_data.shape}")
    
    # Select the first model for analysis
    first_model = pdb_data[0]  # Get the first model
    print(f"Working with model 0, which has {len(first_model)} atoms")
    
    # Show first few atoms from the first model
    print("\nFirst 5 atoms from model 0:")
    for i in range(min(5, len(first_model))):
        atom = first_model[i]
        print(f"Atom {i+1}: {atom.atom_name} {atom.res_name} {atom.res_id} - Chain {atom.chain_id}")
    
    # Select atoms based on a condition (e.g., all atoms in chain A) from first model
    chain_a_atoms = first_model[first_model.chain_id == 'A']
    print(f"\nNumber of atoms in chain A (model 0): {len(chain_a_atoms)}")
    
    # Select atoms from chain B
    chain_b_atoms = first_model[first_model.chain_id == 'B']
    print(f"Number of atoms in chain B (model 0): {len(chain_b_atoms)}")
    
else:
    # It's a single model AtomArray
    print(f"Atom array shape: {pdb_data.shape}")
    
    # Show first few atoms
    print("\nFirst 5 atoms:")
    for i in range(min(5, len(pdb_data))):
        atom = pdb_data[i]
        print(f"Atom {i+1}: {atom.atom_name} {atom.res_name} {atom.res_id} - Chain {atom.chain_id}")
    
    # Select atoms based on a condition (e.g., all atoms in chain A)
    chain_a_atoms = pdb_data[pdb_data.chain_id == 'A']
    print(f"\nNumber of atoms in chain A: {len(chain_a_atoms)}")

# Save the CA atoms in chain A into a array
ca_atoms_chain_a = chain_a_atoms[chain_a_atoms.atom_name == 'CA']
print(f"\nNumber of CA atoms in chain A: {len(ca_atoms_chain_a)}")

print(ca_atoms_chain_a)

# Determine which atoms to work with based on structure type
if isinstance(pdb_data, structure.AtomArrayStack):
    working_atoms = first_model
    chain_b_atoms = first_model[first_model.chain_id == 'B']
else:
    working_atoms = pdb_data
    chain_b_atoms = pdb_data[pdb_data.chain_id == 'B']

# Save all atoms from Chain B
print(f"\nSaving all atoms from Chain B...")
print(f"Number of atoms in Chain B: {len(chain_b_atoms)}")

if len(chain_b_atoms) > 0:
    # Display information about Chain B atoms
    print("\nChain B atom details:")
    for i in range(min(10, len(chain_b_atoms))):  # Show first 10 atoms
        atom = chain_b_atoms[i]
        print(f"  Atom {i+1}: {atom.atom_name} in {atom.res_name} {atom.res_id}")
    
    if len(chain_b_atoms) > 10:
        print(f"  ... and {len(chain_b_atoms) - 10} more atoms")
    
    # Get coordinates of all Chain B atoms
    chain_b_coordinates = chain_b_atoms.coord
    print(f"\nChain B coordinates shape: {chain_b_coordinates.shape}")
    print(f"First 3 coordinates:\n{chain_b_coordinates[:3]}")
else:
    print("No atoms found in Chain B")

# Calculate the centre of mass of the atoms in Chain B (fixed function)
def calculate_center_of_mass(atoms):
    """Calculate the center of mass of a given set of atoms."""
    if len(atoms) == 0:
        return None
    positions = atoms.coord  # Use .coord instead of .get_positions()
    return np.mean(positions, axis=0)

center_of_mass_chain_b = calculate_center_of_mass(chain_b_atoms)
if center_of_mass_chain_b is not None:
    print(f"\nCenter of mass of chain B: {center_of_mass_chain_b}")
else:
    print("\nNo atoms in chain B to calculate center of mass.")

# Calculate the distances between the CA atoms in chain A and the center of mass of chain B
def calculate_distances_to_com(ca_atoms, com):
    """Calculate distances from CA atoms to a given center of mass."""
    if com is None or len(ca_atoms) == 0:
        return None
    ca_positions = ca_atoms.coord  # Use .coord instead of .get_positions()
    distances = np.linalg.norm(ca_positions - com, axis=1)
    return distances

distances_to_com = calculate_distances_to_com(ca_atoms_chain_a, center_of_mass_chain_b)

print(distances_to_com.shape)



PDB Structure Information:
Type of structure: <class 'biotite.structure.AtomArrayStack'>
Number of atoms: 1
Number of models: 1
Atom array shape: (1, 2312)
Working with model 0, which has 2312 atoms

First 5 atoms from model 0:
Atom 1: N ALA 1 - Chain A
Atom 2: CA ALA 1 - Chain A
Atom 3: C ALA 1 - Chain A
Atom 4: O ALA 1 - Chain A
Atom 5: CB ALA 1 - Chain A

Number of atoms in chain A (model 0): 2276
Number of atoms in chain B (model 0): 36

Number of CA atoms in chain A: 273
    A       1  ALA CA     C         4.716  -27.051    2.229
    A       2  TYR CA     C         5.424  -23.401    1.377
    A       3  VAL CA     C         1.948  -22.157    2.284
    A       4  LEU CA     C         3.021  -18.551    2.882
    A       5  TYR CA     C         4.952  -18.537   -0.414
    A       6  TYR CA     C         1.869  -19.527   -2.421
    A       7  LEU CA     C        -0.281  -17.192   -0.305
    A       8  ALA CA     C         1.906  -14.316   -1.498
    A       9  ILE CA     C         1.5

In [5]:
# Need to refactor this so the function works with its input being a single PDB file.

pdb_input = "boltz_results_391/predictions/391/391_model_0.pdb"


def calculate_hydrogen_bonds(chain_a_atoms, chain_b_atoms, distance_cutoff=3.5, angle_cutoff=120):
    """
    Calculate potential hydrogen bonds between Chain A and Chain B based on heavy atom distances
    and estimated angles. Since explicit hydrogens are not present, we approximate geometry using
    bonded atoms.
    
    Parameters:
    -----------
    chain_a_atoms : AtomArray
        Atoms from Chain A (protein)
    chain_b_atoms : AtomArray
        Atoms from Chain B (ligand)
    distance_cutoff : float, default=3.5
        Maximum distance (in Angstroms) between donor and acceptor heavy atoms
    angle_cutoff : float, default=120
        Minimum angle (in degrees) for hydrogen bond geometry (approximated)
    
    Returns:
    --------
    tuple: (hbonds, bond_info)
        hbonds: list of potential hydrogen bonds
        bond_info: detailed information about each bond
    """
    
    # Define potential hydrogen bond donors and acceptors (heavy atoms only)
    donor_atoms = ['N', 'O']  # Atoms that can donate hydrogen
    acceptor_atoms = ['N', 'O', 'S', 'F']  # Atoms that can accept hydrogen
    
    # Get donor and acceptor atoms from each chain
    chain_a_donors = chain_a_atoms[np.isin(chain_a_atoms.element, donor_atoms)]
    chain_a_acceptors = chain_a_atoms[np.isin(chain_a_atoms.element, acceptor_atoms)]
    
    chain_b_donors = chain_b_atoms[np.isin(chain_b_atoms.element, donor_atoms)]
    chain_b_acceptors = chain_b_atoms[np.isin(chain_b_atoms.element, acceptor_atoms)]
    
    print(f"Chain A donors (N,O): {len(chain_a_donors)}")
    print(f"Chain A acceptors (N,O,S,F): {len(chain_a_acceptors)}")
    print(f"Chain B donors (N,O): {len(chain_b_donors)}")
    print(f"Chain B acceptors (N,O,S,F): {len(chain_b_acceptors)}")
    
    def find_bonded_atom(target_atom, atom_array, bond_distance=1.8):
        """Find a bonded atom to estimate hydrogen position"""
        distances = np.linalg.norm(atom_array.coord - target_atom.coord, axis=1)
        bonded_indices = np.where((distances > 0) & (distances <= bond_distance))[0]
        if len(bonded_indices) > 0:
            return atom_array[bonded_indices[0]]  # Return first bonded atom
        return None
    
    def calculate_angle(atom1_coord, atom2_coord, atom3_coord):
        """Calculate angle between three points (atom2 is the vertex)"""
        vec1 = atom1_coord - atom2_coord
        vec2 = atom3_coord - atom2_coord
        
        # Normalize vectors
        vec1_norm = vec1 / np.linalg.norm(vec1)
        vec2_norm = vec2 / np.linalg.norm(vec2)
        
        # Calculate angle
        cos_angle = np.clip(np.dot(vec1_norm, vec2_norm), -1.0, 1.0)
        angle = np.degrees(np.arccos(cos_angle))
        return angle
    
    hydrogen_bonds = []
    bond_info = []
    
    # Check Chain A donors to Chain B acceptors
    for donor in chain_a_donors:
        for acceptor in chain_b_acceptors:
            distance = np.linalg.norm(donor.coord - acceptor.coord)
            
            if distance <= distance_cutoff:
                # Find a bonded atom to donor to estimate hydrogen position
                bonded_to_donor = find_bonded_atom(donor, chain_a_atoms)
                
                if bonded_to_donor is not None:
                    # Calculate angle: bonded_atom - donor - acceptor
                    angle = calculate_angle(bonded_to_donor.coord, donor.coord, acceptor.coord)
                    
                    if angle >= angle_cutoff:
                        hydrogen_bonds.append((donor, acceptor))
                        bond_info.append({
                            'donor_atom': donor.atom_name,
                            'donor_res': donor.res_name,
                            'donor_res_id': donor.res_id,
                            'donor_chain': donor.chain_id,
                            'acceptor_atom': acceptor.atom_name,
                            'acceptor_res': acceptor.res_name,
                            'acceptor_res_id': acceptor.res_id,
                            'acceptor_chain': acceptor.chain_id,
                            'distance': distance,
                            'angle': angle,
                            'donor_coord': donor.coord,
                            'acceptor_coord': acceptor.coord
                        })
                else:
                    # If no bonded atom found, accept based on distance only
                    hydrogen_bonds.append((donor, acceptor))
                    bond_info.append({
                        'donor_atom': donor.atom_name,
                        'donor_res': donor.res_name,
                        'donor_res_id': donor.res_id,
                        'donor_chain': donor.chain_id,
                        'acceptor_atom': acceptor.atom_name,
                        'acceptor_res': acceptor.res_name,
                        'acceptor_res_id': acceptor.res_id,
                        'acceptor_chain': acceptor.chain_id,
                        'distance': distance,
                        'angle': None,
                        'donor_coord': donor.coord,
                        'acceptor_coord': acceptor.coord
                    })
    
    # Check Chain B donors to Chain A acceptors
    for donor in chain_b_donors:
        for acceptor in chain_a_acceptors:
            distance = np.linalg.norm(donor.coord - acceptor.coord)
            
            if distance <= distance_cutoff:
                bonded_to_donor = find_bonded_atom(donor, chain_b_atoms)
                
                if bonded_to_donor is not None:
                    angle = calculate_angle(bonded_to_donor.coord, donor.coord, acceptor.coord)
                    
                    if angle >= angle_cutoff:
                        hydrogen_bonds.append((donor, acceptor))
                        bond_info.append({
                            'donor_atom': donor.atom_name,
                            'donor_res': donor.res_name,
                            'donor_res_id': donor.res_id,
                            'donor_chain': donor.chain_id,
                            'acceptor_atom': acceptor.atom_name,
                            'acceptor_res': acceptor.res_name,
                            'acceptor_res_id': acceptor.res_id,
                            'acceptor_chain': acceptor.chain_id,
                            'distance': distance,
                            'angle': angle,
                            'donor_coord': donor.coord,
                            'acceptor_coord': acceptor.coord
                        })
                else:
                    hydrogen_bonds.append((donor, acceptor))
                    bond_info.append({
                        'donor_atom': donor.atom_name,
                        'donor_res': donor.res_name,
                        'donor_res_id': donor.res_id,
                        'donor_chain': donor.chain_id,
                        'acceptor_atom': acceptor.atom_name,
                        'acceptor_res': acceptor.res_name,
                        'acceptor_res_id': acceptor.res_id,
                        'acceptor_chain': acceptor.chain_id,
                        'distance': distance,
                        'angle': None,
                        'donor_coord': donor.coord,
                        'acceptor_coord': acceptor.coord
                    })
    
    return hydrogen_bonds, bond_info

def analyze_hydrogen_bonds(chain_a_atoms, chain_b_atoms, distance_cutoff=3.5):
    """
    Analyze and display hydrogen bond information between two chains using heavy atoms only.
    """
    hbonds, bond_details = calculate_hydrogen_bonds(
        chain_a_atoms, chain_b_atoms, distance_cutoff
    )
    
    print(f"\nFound {len(hbonds)} potential hydrogen bonds (heavy atom distance < {distance_cutoff} Å)")
    
    if len(hbonds) > 0:
        print("\nPotential hydrogen bond details:")
        for i, bond in enumerate(bond_details):
            print(f"  Bond {i+1}:")
            print(f"    Donor: {bond['donor_atom']} in {bond['donor_res']}{bond['donor_res_id']} (Chain {bond['donor_chain']})")
            print(f"    Acceptor: {bond['acceptor_atom']} in {bond['acceptor_res']}{bond['acceptor_res_id']} (Chain {bond['acceptor_chain']})")
            print(f"    Distance: {bond['distance']:.2f} Å")
            if bond.get('angle') is not None:
                print(f"    Angle: {bond['angle']:.2f}°")
            print()
        
        # Summary statistics
        distances = [bond['distance'] for bond in bond_details]
        print(f"Distance statistics:")
        print(f"  Mean: {np.mean(distances):.2f} Å")
        print(f"  Min: {np.min(distances):.2f} Å")
        print(f"  Max: {np.max(distances):.2f} Å")
        
        # Count by residue types
        donor_residues = [bond['donor_res'] for bond in bond_details]
        acceptor_residues = [bond['acceptor_res'] for bond in bond_details]
        
        unique_donors, donor_counts = np.unique(donor_residues, return_counts=True)
        unique_acceptors, acceptor_counts = np.unique(acceptor_residues, return_counts=True)
        
        print(f"\nDonor residue types: {dict(zip(unique_donors, donor_counts))}")
        print(f"Acceptor residue types: {dict(zip(unique_acceptors, acceptor_counts))}")
    
    return len(hbonds), bond_details

# Calculate hydrogen bonds between Chain A and Chain B using heavy atoms only
if len(chain_a_atoms) > 0 and len(chain_b_atoms) > 0:
    num_hbonds, hbond_details = analyze_hydrogen_bonds(chain_a_atoms, chain_b_atoms, distance_cutoff=3.5)
    print(f"\nTotal potential inter-chain hydrogen bonds: {num_hbonds}")
    
    # Optionally, you can also try different distance cutoffs
    #print(f"\nTrying different distance cutoffs:")
    #for cutoff in [3.0, 3.5, 4.0]:
        #num_bonds, _ = analyze_hydrogen_bonds_heavy_atoms(chain_a_atoms, chain_b_atoms, distance_cutoff=cutoff)
        #print(f"  Distance cutoff {cutoff} Å: {num_bonds} bonds")
else:
    print("Cannot calculate hydrogen bonds: missing atoms in one or both chains")

Chain A donors (N,O): 734
Chain A acceptors (N,O,S,F): 752
Chain B donors (N,O): 12
Chain B acceptors (N,O,S,F): 14

Found 3 potential hydrogen bonds (heavy atom distance < 3.5 Å)

Potential hydrogen bond details:
  Bond 1:
    Donor: ND2 in ASN50 (Chain A)
    Acceptor: N29 in LIG1 (Chain B)
    Distance: 2.99 Å
    Angle: 165.42°

  Bond 2:
    Donor: ND2 in ASN50 (Chain A)
    Acceptor: N33 in LIG1 (Chain B)
    Distance: 2.83 Å
    Angle: 167.06°

  Bond 3:
    Donor: N29 in LIG1 (Chain B)
    Acceptor: ND2 in ASN50 (Chain A)
    Distance: 2.99 Å
    Angle: 120.67°

Distance statistics:
  Mean: 2.93 Å
  Min: 2.83 Å
  Max: 2.99 Å

Donor residue types: {'ASN': 2, 'LIG': 1}
Acceptor residue types: {'ASN': 1, 'LIG': 2}

Total potential inter-chain hydrogen bonds: 3


In [6]:
import sys
import os
import biotite
import biotite.structure as structure
import biotite.structure.io.pdb as pdb
import numpy as np


# Test notebook for Filtering and PDB manipulation

pdb_file = "boltz_results_391/predictions/391/391_model_0.pdb"

# Load the PDB file and parse it
pdb_file_obj = pdb.PDBFile.read(pdb_file)
pdb_data = pdb_file_obj.get_structure()

# Print the structure information
print("PDB Structure Information:")
print(f"Type of structure: {type(pdb_data)}")
print(f"Number of atoms: {len(pdb_data)}")

# Check if it's an AtomArrayStack (multiple models) or AtomArray (single model)
if isinstance(pdb_data, structure.AtomArrayStack):
    print(f"Number of models: {pdb_data.stack_depth()}")
    print(f"Atom array shape: {pdb_data.shape}")
    
    # Select the first model for analysis
    first_model = pdb_data[0]  # Get the first model
        
    
    # Select atoms based on a condition (e.g., all atoms in chain A) from first model
    chain_a_atoms = first_model[first_model.chain_id == 'A']
    print(f"\nNumber of atoms in chain A (model 0): {len(chain_a_atoms)}")
    
    # Select atoms from chain B
    chain_b_atoms = first_model[first_model.chain_id == 'B']
    print(f"Number of atoms in chain B (model 0): {len(chain_b_atoms)}")
    
else:
    # It's a single model AtomArray
    print(f"Atom array shape: {pdb_data.shape}")
    
    # Show first few atoms
    print("\nFirst 5 atoms:")
    for i in range(min(5, len(pdb_data))):
        atom = pdb_data[i]
        print(f"Atom {i+1}: {atom.atom_name} {atom.res_name} {atom.res_id} - Chain {atom.chain_id}")
    
    # Select atoms based on a condition (e.g., all atoms in chain A)
    chain_a_atoms = pdb_data[pdb_data.chain_id == 'A']
    print(f"\nNumber of atoms in chain A: {len(chain_a_atoms)}")

# Save the CA atoms in chain A into a array
ca_atoms_chain_a = chain_a_atoms[chain_a_atoms.atom_name == 'CA']
print(f"\nNumber of CA atoms in chain A: {len(ca_atoms_chain_a)}")

print(ca_atoms_chain_a)

# Determine which atoms to work with based on structure type
if isinstance(pdb_data, structure.AtomArrayStack):
    working_atoms = first_model
    chain_b_atoms = first_model[first_model.chain_id == 'B']
else:
    working_atoms = pdb_data
    chain_b_atoms = pdb_data[pdb_data.chain_id == 'B']

# Save all atoms from Chain B
print(f"\nSaving all atoms from Chain B...")
print(f"Number of atoms in Chain B: {len(chain_b_atoms)}")

if len(chain_b_atoms) > 0:
    # Display information about Chain B atoms
    print("\nChain B atom details:")
    for i in range(min(10, len(chain_b_atoms))):  # Show first 10 atoms
        atom = chain_b_atoms[i]
        print(f"  Atom {i+1}: {atom.atom_name} in {atom.res_name} {atom.res_id}")
    
    if len(chain_b_atoms) > 10:
        print(f"  ... and {len(chain_b_atoms) - 10} more atoms")
    
    # Get coordinates of all Chain B atoms
    chain_b_coordinates = chain_b_atoms.coord
    print(f"\nChain B coordinates shape: {chain_b_coordinates.shape}")
    print(f"First 3 coordinates:\n{chain_b_coordinates[:3]}")
else:
    print("No atoms found in Chain B")

PDB Structure Information:
Type of structure: <class 'biotite.structure.AtomArrayStack'>
Number of atoms: 1
Number of models: 1
Atom array shape: (1, 2312)

Number of atoms in chain A (model 0): 2276
Number of atoms in chain B (model 0): 36

Number of CA atoms in chain A: 273
    A       1  ALA CA     C         4.716  -27.051    2.229
    A       2  TYR CA     C         5.424  -23.401    1.377
    A       3  VAL CA     C         1.948  -22.157    2.284
    A       4  LEU CA     C         3.021  -18.551    2.882
    A       5  TYR CA     C         4.952  -18.537   -0.414
    A       6  TYR CA     C         1.869  -19.527   -2.421
    A       7  LEU CA     C        -0.281  -17.192   -0.305
    A       8  ALA CA     C         1.906  -14.316   -1.498
    A       9  ILE CA     C         1.596  -15.352   -5.155
    A      10  VAL CA     C        -2.180  -15.814   -4.912
    A      11  GLY CA     C        -2.654  -12.614   -2.926
    A      12  HIS CA     C        -0.628  -10.417   -5.264
   

In [7]:
def extract_ca_positions(pdb_data, chain_id):
    """
    Extract CA positions from a specified chain in the PDB data.

    Parameters:
    -----------
    pdb_data : AtomArray or AtomArrayStack
        The PDB structure data.
    chain_id : str
        The chain ID to extract CA positions from.

    Returns:
    --------
    AtomArray
        The CA atoms from the specified chain.
    """
    if isinstance(pdb_data, structure.AtomArrayStack):
        pdb_data = pdb_data[0]  # Use the first model if it's an AtomArrayStack
    chain_atoms = pdb_data[pdb_data.chain_id == chain_id]
    ca_atoms = chain_atoms[chain_atoms.atom_name == 'CA']
    return ca_atoms


def extract_atoms_by_chain(pdb_data, chain_id):
    """
    Extract all atoms from a specified chain in the PDB data.

    Parameters:
    -----------
    pdb_data : AtomArray or AtomArrayStack
        The PDB structure data.
    chain_id : str
        The chain ID to extract atoms from.

    Returns:
    --------
    AtomArray
        All atoms from the specified chain.
    """
    if isinstance(pdb_data, structure.AtomArrayStack):
        pdb_data = pdb_data[0]  # Use the first model if it's an AtomArrayStack
    chain_atoms = pdb_data[pdb_data.chain_id == chain_id]
    return chain_atoms


# Example usage:
pdb_file = "boltz_results_391/predictions/391/391_model_0.pdb"
pdb_file_obj = pdb.PDBFile.read(pdb_file)
pdb_data = pdb_file_obj.get_structure()

# Extract CA positions from chain A
ca_atoms_chain_a = extract_ca_positions(pdb_data, chain_id='A')
print(f"\nNumber of CA atoms in chain A: {len(ca_atoms_chain_a)}")
print(ca_atoms_chain_a)

# Extract all atoms from chain B
chain_b_atoms = extract_atoms_by_chain(pdb_data, chain_id='B')
print(f"\nNumber of atoms in chain B: {len(chain_b_atoms)}")
if len(chain_b_atoms) > 0:
    print(chain_b_atoms)
else:
    print("No atoms found in Chain B")




Number of CA atoms in chain A: 273
    A       1  ALA CA     C         4.716  -27.051    2.229
    A       2  TYR CA     C         5.424  -23.401    1.377
    A       3  VAL CA     C         1.948  -22.157    2.284
    A       4  LEU CA     C         3.021  -18.551    2.882
    A       5  TYR CA     C         4.952  -18.537   -0.414
    A       6  TYR CA     C         1.869  -19.527   -2.421
    A       7  LEU CA     C        -0.281  -17.192   -0.305
    A       8  ALA CA     C         1.906  -14.316   -1.498
    A       9  ILE CA     C         1.596  -15.352   -5.155
    A      10  VAL CA     C        -2.180  -15.814   -4.912
    A      11  GLY CA     C        -2.654  -12.614   -2.926
    A      12  HIS CA     C        -0.628  -10.417   -5.264
    A      13  SER CA     C        -2.021  -11.992   -8.451
    A      14  LEU CA     C        -5.560  -11.458   -7.128
    A      15  SER CA     C        -4.581   -7.893   -6.214
    A      16  ILE CA     C        -3.347   -7.250   -9.773
    